In [1]:
import os
import sys
import math
import logging
import yaml
import h5py
from pathlib import Path

import kerastuner as kt

import numpy as np
import scipy as sp
import tensorflow as tf

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import began
from began.logging import setup_vae_run_logging

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
class CVAE(tf.keras.Model):
    def __init__(self, latent_dim, kernel_size):
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        self.inference_net = make_vae_inference_net(latent_dim)
        self.generative_net = make_vae_generative_net(latent_dim, kernel_size)

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        mean, logvar = tf.split(self.inference_net(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.generative_net(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

In [4]:
class DustVAEder(object):
    """ Convolutional variational autoencoder model
    """
    def __init__(self, hyperparameters):
        self.hps = hyperparameters
        self.model = self._build_model()
        self.model.summary()

        # Optimizer
        self.optimizer = Adam(beta_1=self.hps['beta_1'], lr=self.hps['learning_rate'])

        # Loss function
        self.loss = self._model_loss()
        
        # Compile model
        self.model.compile(optimizer=self.optimizer, loss=self.loss)

    def _build_model(self):
        inputs = 
        decoder = build_decoder(self.hps['latent_dim'], self.hps['kernel_size'])
        encoder = build_encoder(self.hps['latent_dim'])
        tf.keras.models.Model()
        return

    def _model_loss(self):
        rec_loss = self._reconstruction_loss()
        kl_loss = self._kl_divergence_lsos()

        @tf.function
        def _total_loss(x_true, x_pred):
            rec_loss(x_true, x_pred)
            return rec_loss + kl_loss
        return _total_loss

    def _reconstruction_loss(self):
        return

    def _kl_divergence_loss(self):
        return

@tf.function
def log_normal_pdf(sample, mean, logvar, raxis=1):\
    """ Convenience function to calculate lognormal pdf.
    """
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)

def build_decoder(latent_dim, kernel_size):
    model = tf.keras.Sequential(name='Decoder')
    
    model.add(tf.keras.layers.InputLayer(input_shape=(latent_dim,)))
    model.add(tf.keras.layers.Dense(units=16*16*32, activation=tf.nn.relu))
    model.add(tf.keras.layers.Reshape(target_shape=(16, 16, 32)))
    assert model.output_shape == (None, 16, 16, 32)
    
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=kernel_size, strides=(2, 2), padding="SAME", activation='relu'))
    assert model.output_shape == (None, 32, 32, 128)
    
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=kernel_size, strides=(2, 2), padding="SAME", activation='relu'))
    assert model.output_shape == (None, 64, 64, 128)
    
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=kernel_size, strides=(2, 2), padding="SAME", activation='relu'))
    assert model.output_shape == (None, 128, 128, 128)
    
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=kernel_size, strides=(2, 2), padding="SAME", activation='relu'))
    assert model.output_shape == (None, 256, 256, 128)
    
    model.add(tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=kernel_size, strides=(1, 1), padding="SAME"))
    return model


def build_encoder(latent_dim):
    model = tf.keras.Sequential(name='Encoder')
    
    model.add(tf.keras.layers.InputLayer(input_shape=(256, 256, 1)))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding="SAME", strides=(2, 2), activation='relu'))
    assert model.output_shape == (None, 128, 128, 256)
    
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="SAME", strides=(2, 2), activation='relu'))
    assert model.output_shape == (None, 64, 64, 128)
    
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=1, padding="SAME", strides=(2, 2), activation='relu'))
    assert model.output_shape == (None, 32, 32, 64)
    
    model.add(tf.keras.layers.Flatten())
    assert model.output_shape == (None, 32 * 32 * 64)
    
    model.add(tf.keras.layers.Dense(latent_dim + latent_dim))
    return model
    
    

In [3]:
# initialize random seed in numpy
np.random.seed(123454321)
# initialize random seed in tensorflow
tf.random.set_seed(123454321)

In [4]:
# Batch and shuffle the data
with h5py.File("../data/preprocessed/prepared.h5", 'r') as f:
    dset = f["cut_maps"]
    train_images = dset[...].astype(np.float32)

dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_images.shape[0]).map(tf.image.per_image_standardization)
test_dataset = dataset.take(100)
train_dataset = dataset.skip(100)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
def build_model(hp):
    """Builds a convolutional model."""
    lat_dim = hp.Int('lat_dim', 32, 256)
    kernel_size = hp.Choice('kernel_size', values=[3, 5])
    return began.CVAE(lat_dim, kernel_size)

In [6]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, train_ds):
        hp = trial.hyperparameters

        # Hyperparameters can be added anywhere inside `run_trial`.
        # When the first trial is run, they will take on their default values.
        # Afterwards, they will be tuned by the `Oracle`.
        train_ds = train_ds.batch(hp.Int('batch_size', 8, 32, default=8))
        print(type(train_ds))
        model = self.hypermodel.build(trial.hyperparameters)
        lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log', default=1e-3)
        optimizer = tf.keras.optimizers.Adam(beta_1=0.5, learning_rate=lr)
        epoch_loss_metric = tf.keras.metrics.Mean()

        @tf.function
        def run_train_step(data):
            with tf.GradientTape() as tape:
                loss = began.vae.compute_loss(model, data)
                gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            epoch_loss_metric.update_state(loss)
            return loss

        # `self.on_epoch_end` reports results to the `Oracle` and saves the
        # current state of the Model. The other hooks called here only log values
        # for display but can also be overridden. For use cases where there is no
        # natural concept of epoch, you do not have to call any of these hooks. In
        # this case you should instead call `self.oracle.update_trial` and
        # `self.oracle.save_model` manually.
        for epoch in range(3):
            print('Epoch: {}'.format(epoch))

            self.on_epoch_begin(trial, model, epoch, logs={})
            for batch, data in enumerate(train_ds):
                self.on_batch_begin(trial, model, batch, logs={})
                batch_loss = float(run_train_step(data))
                self.on_batch_end(trial, model, batch, logs={'loss': batch_loss})

                if batch % 30 == 0:
                    loss = epoch_loss_metric.result().numpy()
                    print('Batch: {}, Average Loss: {}'.format(batch, loss))

            epoch_loss = epoch_loss_metric.result().numpy()
            self.on_epoch_end(trial, model, epoch, logs={'loss': epoch_loss})
            epoch_loss_metric.reset_states()

In [7]:
tuner = MyTuner(
      oracle=kt.oracles.BayesianOptimization(
          objective=kt.Objective('loss', 'min'),
          max_trials=10),
      hypermodel=build_model,
      directory='results',
      project_name='began_custom_training_hp')

INFO:tensorflow:Reloading Oracle from existing project results/began_custom_training_hp/oracle.json
INFO:tensorflow:Reloading Oracle from existing project results/began_custom_training_hp/oracle.json
INFO:tensorflow:Reloading Tuner from results/began_custom_training_hp/tuner0.json
INFO:tensorflow:Reloading Tuner from results/began_custom_training_hp/tuner0.json


In [13]:
tuner.search(train_ds=train_dataset)

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


In [14]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

{'lat_dim': 131, 'kernel_size': 5, 'batch_size': 10, 'learning_rate': 0.00019071203207567507}


In [15]:
best_model = tuner.get_best_models()[0]